In [3]:
import pandas as pd
import re
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer

c:\Users\s.pardob\OneDrive - Universidad de los Andes\Escritorio\8-semestre\Tesis\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Redes neuronales basdas en grafos

# Entities Exploration

## Concept

In [14]:
concept_df = pd.read_json("../data/MOOCCube/entities/concept.json",lines=True)

In [15]:
concept_df.shape

(114563, 4)

In [16]:
concept_df.sample(5)

,id,name,en,explanation
38288,K_奥地利学派_心理学,奥地利学派,Austrian school,学科：心理学_理论心理学与心理学史 定义：主张心理学研究对象是意动而不是内容，方法是观察而不...
89327,K_非产权战略联盟_管理科学技术,非产权战略联盟,nonequity strategic alliance,学科：管理科学技术_一般管理理论_战略管理 定义：两个或者两个以上的企业通过发展企业间契约关...
69660,K_回潮_材料科学技术,回潮,conditioning,学科：材料科学技术_天然材料 _皮革 定义：使干燥后的皮革重新吸收一定量的水分，从而使皮革中...
55648,K_风压试验_机械工程,风压试验,pressure decay test,学科：机械工程_动力机械_锅炉_试验和测试_试验 定义：按规定的压力和保持时间对炉膛或烟道用...
58883,K_酸性渣_机械工程,酸性渣,acid slag,学科：机械工程_机械制造工艺与设备 _焊接与切割 _焊接材料 定义：化学性质呈酸性的熔渣。 ...


## Course

In [17]:
course_df = pd.read_json("../data/MOOCCube/entities/course.json",lines=True)

In [18]:
course_df.shape

(706, 8)

In [19]:
course_df.sample(5)

,id,name,prerequisites,about,core_id,video_order,display_name,chapter
84,C_course-v1:RiceX+BIOC372_1x+sp,免疫学基础——第Ⅰ部分（自主模式）,入门级生物学知识或相当水平。,<p>\r\n\t当你生病的时候，你可能会感到奇怪，“为什么是我呢？”，但是真正的问题应该是...,C_course-v1:RiceX+BIOC372_1x+sp,"[V_77df216e333344459902972c65d6a59a, V_8d863bd...","[I. Welcome!, II. Staying Healthy, III. ...","[01.02.01.01, 01.02.02.02, 01.02.03.02, 01.02...."
469,C_course-v1:TsinghuaX+00510888X+2019_T1,创业启程（2019春）,本课程为基础课，无先修要求,<p>\n\t采用“理论+案例”相结合的形式，由企业家亲自讲授创业方法或案例，教授点评。在学...,C_course-v1:TsinghuaX+00510888X+2019_T1,"[V_08aeb5d07f594263ad7c2418a15363e2, V_0262a6b...","[Video, 默认, 默认, 默认, 默认, 创业机会的分类, 何谓好的商业机会, 个人创...","[01.01.01.01, 01.02.01.01, 01.03.01.01, 01.04...."
1,C_course-v1:TsinghuaX+THESIS2015X+2015_T1,2015年清华大学研究生学位论文答辩（二）,无先修要求,<p>学位论文答辩环节是研究生培养的重要环节，为了充分发挥该环节的育人作用，搭建学术交流的平...,C_course-v1:TsinghuaX+THESIS2015X+2015_T1,"[V_de0371575a9f4b5391c89ad16d68b5c2, V_d632034...","[答辩陈述, 答辩陈述, 问答及答辩结果, 答辩陈述, 问答及答辩结果, 答辩陈述, 问答及...","[01.01.03.01, 01.02.03.01, 01.02.04.01, 01.03...."
106,C_course-v1:TsinghuaX+01030132X+sp,大学历史与文化（自主模式）,没有知识的要求，只要兴趣的驱动,<p>\r\n\t人类没有大学，世界将会怎样？\r\n</p>\r\n<p>\r\n\t怎样...,C_course-v1:TsinghuaX+01030132X+sp,"[V_691d1f66a4a74b368ee5d48a7517e152, V_50774e4...","[中学生怎么看大学, 园子里的人怎么看大学, 导读：教育的目的--认识名著, 导读：教育的目...","[01.01.01.01, 01.02.01.01, 01.03.01.01, 01.04...."
413,C_course-v1:UIR+2018040201X+2019_T1,大学生国家安全教育（2019春）,无,<p>\r\n\t《大学生国家安全教育》课程具有以下特色：一是课程内容丰富。课程以习近平总体...,C_course-v1:UIR+2018040201X+2019_T1,"[V_cfc5a04fe5b348eaa11c90def13e40ca, V_93aaa75...","[Video, Video, Video, Video, Video, Video, Vid...","[01.01.01.01, 01.01.02.01, 01.01.03.01, 02.01...."


In [20]:
def remove_html_and_carriage_returns(text):
    soup = BeautifulSoup(text, "html.parser")
    text_without_html = soup.get_text().replace("\n","").replace("\r","").replace("\t","")
    
    return text_without_html

course_df["about"] = course_df["about"].apply(remove_html_and_carriage_returns)
course_df.sample(5)

,id,name,prerequisites,about,core_id,video_order,display_name,chapter
480,C_course-v1:TsinghuaX+00690092X+2019_T1,逻辑学概论（2019春）,本课程无先修要求。如果有良好的数学训练基础，将有助于更好地深入领会逻辑学的方法精神，但这并非必须。,“逻辑”一词很常用，但作为一门学科和课程，“逻辑学”涉及的内容范围，比“逻辑”一词日常的用法...,C_course-v1:TsinghuaX+00690092X+sp,"[V_e36603ea30c540ac9aa17893591d75c6, V_47781e1...","[默认, movie_01_02.mp4, movie_01_03.mp4, movie_0...","[01.01.01.01, 01.02.01.01, 01.03.01.01, 01.04...."
430,C_course-v1:HIT+13SC20301820+2019_T1,会计信息系统（2019春）,大学计算机基础、数据库系统、中级财务会计,“会计信息系统”是会计学专业必修的核心课程。课程使学生掌握会计信息系统的基本理论：会计软件的...,C_course-v1:HIT+13SC20301820+2019_T1,"[V_392a3e6051b04a3f9a65a02a847f773e, V_b9799cb...","[《会计信息系统》课程导学, None, None, （001）1.1.1 信息（15分钟）...","[01.01.01.01, 01.02.01.02, 01.02.01.03, 03.01...."
236,C_course-v1:QHU+20180419+sp,丝路南线民间文学（自主模式）,了解丝绸之路的相关知识和青海、甘肃、新疆、四川、西藏等地的民间文学类型与内容。,本课程为人文素质选修课，共有丝路与丝路南线、丝路南线民间文学的基本知识、丝路南线上的多民族神...,C_course-v1:QHU+20180419+sp,"[V_7d557974d55541c4845fd373a3b2f965, V_aa2dd86...","[Video, Video, Video, Video, Video, Video, Vid...","[01.01.01.01, 01.02.01.01, 01.03.01.01, 01.04...."
1,C_course-v1:TsinghuaX+THESIS2015X+2015_T1,2015年清华大学研究生学位论文答辩（二）,无先修要求,学位论文答辩环节是研究生培养的重要环节，为了充分发挥该环节的育人作用，搭建学术交流的平台，进...,C_course-v1:TsinghuaX+THESIS2015X+2015_T1,"[V_de0371575a9f4b5391c89ad16d68b5c2, V_d632034...","[答辩陈述, 答辩陈述, 问答及答辩结果, 答辩陈述, 问答及答辩结果, 答辩陈述, 问答及...","[01.01.03.01, 01.02.03.01, 01.02.04.01, 01.03...."
396,C_course-v1:nxu+2018122711+2018_T2,解读西夏,本课程无须先修知识。,西夏是由党项族建立的多民族政权，虽偏处西北，但却创造了我国历史上一系列辉煌，如世界上最早的活...,C_course-v1:nxu+2018122711+2018_T2,"[V_cf68bfcb40424dfba1eaf30c02e0bb71, V_2a0c302...","[第一节 唐代边疆形势与党项的第一次大迁徙, 第二节 安史之乱与党项第二次大迁徙, 第三节 ...","[01.01.01.01, 01.02.01.01, 01.03.01.01, 01.04...."


In [21]:
# sentences = []
# course_ids = [] 
# for index, row in course_df.iterrows():
#     course_ids.append(row["id"])
#     sentences.append(row["about"])
# model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v1')
# embeddings = model.encode(sentences)


In [22]:
course_df = course_df.explode('video_order').reset_index()
course_df.head(5)

,index,id,name,prerequisites,about,core_id,video_order,display_name,chapter
0,0,C_course-v1:McGillX+ATOC185x+2015_T1,自然灾害（自主模式）,无,地球上没有一处地方不发生自然灾害。当我们以科学的眼光看待这些自然灾害的原因和本质时，我们可以...,C_course-v1:McGillX+ATOC185x+2015_T1,V_f6f710068b994452885b90e11b6ee5c5,"[Video: Overview 1, Video: Overview 2, Video: ...","[01.02.01.02, 01.02.03.02, 01.02.05.02, 01.02...."
1,0,C_course-v1:McGillX+ATOC185x+2015_T1,自然灾害（自主模式）,无,地球上没有一处地方不发生自然灾害。当我们以科学的眼光看待这些自然灾害的原因和本质时，我们可以...,C_course-v1:McGillX+ATOC185x+2015_T1,V_73395684a8e24e2ea378b926214335ea,"[Video: Overview 1, Video: Overview 2, Video: ...","[01.02.01.02, 01.02.03.02, 01.02.05.02, 01.02...."
2,0,C_course-v1:McGillX+ATOC185x+2015_T1,自然灾害（自主模式）,无,地球上没有一处地方不发生自然灾害。当我们以科学的眼光看待这些自然灾害的原因和本质时，我们可以...,C_course-v1:McGillX+ATOC185x+2015_T1,V_8bc91d3fc5914379ae31723cdc0de9f1,"[Video: Overview 1, Video: Overview 2, Video: ...","[01.02.01.02, 01.02.03.02, 01.02.05.02, 01.02...."
3,0,C_course-v1:McGillX+ATOC185x+2015_T1,自然灾害（自主模式）,无,地球上没有一处地方不发生自然灾害。当我们以科学的眼光看待这些自然灾害的原因和本质时，我们可以...,C_course-v1:McGillX+ATOC185x+2015_T1,V_8aef0ca48efc445c9edf545114ba22a9,"[Video: Overview 1, Video: Overview 2, Video: ...","[01.02.01.02, 01.02.03.02, 01.02.05.02, 01.02...."
4,0,C_course-v1:McGillX+ATOC185x+2015_T1,自然灾害（自主模式）,无,地球上没有一处地方不发生自然灾害。当我们以科学的眼光看待这些自然灾害的原因和本质时，我们可以...,C_course-v1:McGillX+ATOC185x+2015_T1,V_ae9b4e594d4347a4b3da95dc1851eac3,"[Video: Overview 1, Video: Overview 2, Video: ...","[01.02.01.02, 01.02.03.02, 01.02.05.02, 01.02...."


In [23]:
course_df = course_df.drop('index', axis=1)
course_df.head(5)

,id,name,prerequisites,about,core_id,video_order,display_name,chapter
0,C_course-v1:McGillX+ATOC185x+2015_T1,自然灾害（自主模式）,无,地球上没有一处地方不发生自然灾害。当我们以科学的眼光看待这些自然灾害的原因和本质时，我们可以...,C_course-v1:McGillX+ATOC185x+2015_T1,V_f6f710068b994452885b90e11b6ee5c5,"[Video: Overview 1, Video: Overview 2, Video: ...","[01.02.01.02, 01.02.03.02, 01.02.05.02, 01.02...."
1,C_course-v1:McGillX+ATOC185x+2015_T1,自然灾害（自主模式）,无,地球上没有一处地方不发生自然灾害。当我们以科学的眼光看待这些自然灾害的原因和本质时，我们可以...,C_course-v1:McGillX+ATOC185x+2015_T1,V_73395684a8e24e2ea378b926214335ea,"[Video: Overview 1, Video: Overview 2, Video: ...","[01.02.01.02, 01.02.03.02, 01.02.05.02, 01.02...."
2,C_course-v1:McGillX+ATOC185x+2015_T1,自然灾害（自主模式）,无,地球上没有一处地方不发生自然灾害。当我们以科学的眼光看待这些自然灾害的原因和本质时，我们可以...,C_course-v1:McGillX+ATOC185x+2015_T1,V_8bc91d3fc5914379ae31723cdc0de9f1,"[Video: Overview 1, Video: Overview 2, Video: ...","[01.02.01.02, 01.02.03.02, 01.02.05.02, 01.02...."
3,C_course-v1:McGillX+ATOC185x+2015_T1,自然灾害（自主模式）,无,地球上没有一处地方不发生自然灾害。当我们以科学的眼光看待这些自然灾害的原因和本质时，我们可以...,C_course-v1:McGillX+ATOC185x+2015_T1,V_8aef0ca48efc445c9edf545114ba22a9,"[Video: Overview 1, Video: Overview 2, Video: ...","[01.02.01.02, 01.02.03.02, 01.02.05.02, 01.02...."
4,C_course-v1:McGillX+ATOC185x+2015_T1,自然灾害（自主模式）,无,地球上没有一处地方不发生自然灾害。当我们以科学的眼光看待这些自然灾害的原因和本质时，我们可以...,C_course-v1:McGillX+ATOC185x+2015_T1,V_ae9b4e594d4347a4b3da95dc1851eac3,"[Video: Overview 1, Video: Overview 2, Video: ...","[01.02.01.02, 01.02.03.02, 01.02.05.02, 01.02...."


## Papers

In [24]:
paper_df = pd.read_json("../data/MOOCCube/entities/paper.json",lines=True)

In [25]:
paper_df.shape

(679790, 14)

In [26]:
paper_df.sample(5)

,id,name,abstract,authors,doi,lang,num_citation,pdf,sourcetype,title,venue,year,pages,urls
425846,P_53e9af99b7602d97039e3d61,,,"[{'id': '53f437f4dabfaeee229b9498', 'name': 'H...",10.1112/jlms/s1-34.1.93,en,4,NaN,publication,On the Limits of Riemann Sums,{'info': {'name': 'Journal of The London Mathe...,1959,"{'end': '100', 'start': '93'}",[http://dx.doi.org/10.1112/jlms/s1-34.1.93]
509855,P_56d8da30dabfae2eeee766d1,,,"[{'id': '53f46ae2dabfaee4dc86036c', 'name': 'h...",10.1063/1.1566438,en,97,NaN,publication,Mechanism for ultrafast internal conversion of...,{'info': {'name': 'Journal of Chemical Physics...,2003,"{'end': '', 'start': ''}",[http://dx.doi.org/10.1063/1.1566438]
455111,P_56d889cddabfae2eeeac93b8,,,"[{'name': 'f h tobins'}, {'name': 'ain onuoha'}]",10.4314/jmdmes.v1i1.19016,en,0,NaN,publication,The Vibration Effect Of Unbalance Mass On Vert...,"{'issue': '', 'volume': ''}",2002,"{'end': '', 'start': ''}",[http://dx.doi.org/10.4314/jmdmes.v1i1.19016]
232125,P_56d85a4cdabfae2eee494b23,,,"[{'id': '53f32db6dabfae9a8449c21c', 'name': 'j...",,en,4,NaN,publication,Deaerating feed-water heater,"{'issue': '', 'volume': ''}",1951,"{'end': '', 'start': ''}",NaN
267844,P_5c756ee7f56def979861ade5,,NaN,"[{'name': 'Jed Ludlow'}, {'name': 'T.D. Willia...",,en,0,NaN,publication,Statistical Tolerance Intervals: A Better Appr...,"{'info': {'name': 'Corrosion'}, 'issue': '', '...",2012,"{'end': '', 'start': ''}",NaN


## School

In [27]:
school_df = pd.read_json("../data/MOOCCube/entities/school.json",lines=True)

In [28]:
school_df.shape

(208, 3)

In [29]:
school_df.sample(5)

,id,name,about
167,S_FJNU,福建师范大学,福建师范大学（Fujian Normal University）是福建省人民政府与中华人民共...
200,S_cqu,重庆大学,重庆大学（Chongqing University），简称重大（CQU），位于中央直辖市重庆...
163,S_AFEU,空军工程大学,中国人民解放军空军工程大学（Air Force Engineering University...
191,S_NWPU,西北工业大学,西北工业大学（Northwestern Polytechnical University）简...
124,S_ZZU,郑州大学,郑州大学（Zhengzhou University），简称“郑大”，是中华人民共和国教育部与...


## Teacher

In [30]:
teacher_df = pd.read_json("../data/MOOCCube/entities/teacher.json",lines=True)

In [31]:
teacher_df.shape

(1738, 3)

In [32]:
teacher_df.sample(5)

,id,name,about
1484,T_马成功,马成功,乐视培训发展总经理，原京东大学校长
537,T_李江玲,李江玲,李江玲，人物形象设计专业科班生，高级形象设计师、高级化妆师、高级美容师、IFBC韩国美容协会...
191,T_张平,张平,"张平, 女，1980年4月出生，天津人，中国共产党党员，经济学博士，教授，博士生导师，天津市..."
287,T_邓俊辉,邓俊辉,清华大学博士，学生尊称其为“邓公”，清华大学教学领域最高荣誉「新百年教学成就奖」首届获得者，...
1683,T_任艳频,任艳频,分别于1995年、1999年在清华大学自动化系获得工学学士和工学博士学位。\n1999-20...


## User

In [8]:
user_df = pd.read_json("../data/MOOCCube/entities/user.json",lines=True)

In [9]:
user_df.shape

(199199, 4)

In [6]:
user_df.sample(5)

97787     [C_course-v1:TsinghuaX+20250064+sp, C_course-v...
142370    [C_course-v1:TsinghuaX+2018031601X+2019_T1, C_...
8264      [C_course-v1:TsinghuaX+70612463X+2019_T1, C_co...
126224    [C_course-v1:TsinghuaX+30640014X+2019_T1, C_co...
144628    [C_course-v1:TsinghuaX+20250064+sp, C_course-v...
Name: course_order, dtype: object

In [16]:
i = 0
for index, row in user_df.iterrows():
    if len(row["course_order"])>7 and i<3:
        i +=1
        print(row["course_order"])
    

['C_course-v1:TsinghuaX+20430064_2X+sp', 'C_course-v1:TsinghuaX+02070251X+2019_T1', 'C_course-v1:TsinghuaX+0350161X_2015_T2+sp', 'C_course-v1:TsinghuaX+60240013X+sp', 'C_course-v1:TsinghuaX+01030132X+sp', 'C_course-v1:TsinghuaX+20250103X+sp', 'C_course-v1:TsinghuaX+20220214X+2019_T1', 'C_course-v1:TsinghuaX+02070251X+sp', 'C_course-v1:TsinghuaX+20220053X_2015_T2+sp', 'C_course-v1:TsinghuaX+20350033X+sp']
['C_course-v1:TsinghuaX+30640014X+2019_T1', 'C_course-v1:TsinghuaX+00612643X+sp', 'C_course-v1:TsinghuaX+00612642X+2019_T1', 'C_course-v1:TsinghuaX+20250103X+sp', 'C_course-v1:TsinghuaX+30640014X+sp', 'C_course-v1:TsinghuaX+00701032X+sp', 'C_course-v1:HNU+GE14068+2019_T1', 'C_course-v1:HEBNU+2017011901X+sp', 'C_course-v1:TsinghuaX+00612642X+sp']
['C_course-v1:gmc+3060511+2018_T2', 'C_course-v1:WageningenX+NUTR101x+sp', 'C_course-v1:TsinghuaX+80000271X+sp', 'C_course-v1:TsinghuaX+00040132X+sp', 'C_course-v1:MIL+MIL2016001+sp', 'C_course-v1:UQx+BIOIMG101x+sp', 'C_course-v1:PSFFC+20181024

KeyboardInterrupt: 

## Video

In [36]:
video_df = pd.read_json("../data/MOOCCube/entities/video.json",lines=True)

In [37]:
video_df.shapet

(38181, 5)

In [38]:
video_df.sample(5)

,id,name,start,end,text
35846,V_068f4172eaf74038b76e1df4dff4fcc2,病例对照研究设计与实施,"[14600, 15600, 18986, 20550, 22975, 24851, 283...","[15600, 18802, 20550, 22825, 24851, 26850, 304...","[大家好, 我是中南大学湘雅公共卫生学院的李杏莉, 今天我们要介绍的是, 病例对照研究的设计..."
29428,V_459e23b64d0845e8af212cc806c5bb92,9.4 极限与配合,"[17120, 18060, 21760, 23940, 27780, 29580, 333...","[18060, 21760, 23320, 27780, 29580, 33000, 375...","[同学们好, 这一节我们学习零件图中的技术要求, 极限与配合, 极限与配合是零件图和装配图中..."
4816,V_d771400665e747498e669e417d54c02e,Video,"[20596, 22498, 25119, 30262, 36919, 40250, 424...","[22298, 24919, 30062, 33229, 40050, 42221, 441...","[大家好, 生态系统是这门课的重点, 这一讲主要介绍森林生态系统和湿地生态系统, 我们首先来..."
36505,V_c1833820d0ef4e68bac788d12449ce15,Video_CHAPTER 32 硅铍石,"[15000, 21760, 26360, 31080, 33320, 37080, 394...","[21720, 26320, 31040, 33280, 37040, 39440, 427...","[下一种硅铍石 矿物名称就是硅铍石, 它是一种铍的硅酸盐, 硅铍石的形态呢通常是柱状的, 也..."
21461,V_ec692d4cd3a34bdc9313063d5d1c12f3,六、人物写生-6.2 铅笔写生-单元-Video,NaN,NaN,NaN
